In [ ]:
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

plt.style.use('style.mplstyle')

def fechar_plot(directory, filename, xlabel='Época/Iteração', ylabel='Valor'):
    """Salva o gráfico atual e o fecha, garantindo o tight layout."""
    plots_dir = os.path.join(directory, 'plots')
    os.makedirs(plots_dir, exist_ok=True)
    
    # Define o sufixo do título apenas para métricas que usam Média e STD
    title_suffix = ' (Média ± STD)' if any(s in filename for s in ['loss', 'grad', 'weight']) else ''
    plt.title(f'{filename.replace("_", " ").title()}{title_suffix}')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    if plt.gca().get_lines() or plt.gca().collections:
         plt.legend() 
    
    try:
        plt.tight_layout()
    except Exception:
        pass 
        
    plt.savefig(os.path.join(plots_dir, f'{filename}.pdf'), bbox_inches='tight')
    plt.close()

def coletar_dados_experimentos(caminho_base):
    """
    Varre as subpastas, lê 'resultados.csv', concatena e salva o arquivo compilado.
    Retorna o DataFrame compilado.
    """
    dados_totais = []
    
    try:
        conteudo_pasta = os.listdir(caminho_base)
    except FileNotFoundError:
        print(f"Erro: O caminho base '{caminho_base}' não foi encontrado.")
        return None    
    
    for nome_pasta in conteudo_pasta:
        caminho_experimento = os.path.join(caminho_base, nome_pasta)

        if os.path.isdir(caminho_experimento):
            caminho_csv = os.path.join(caminho_experimento, 'resultados.csv')
            if os.path.exists(caminho_csv):
                try:
                    df = pd.read_csv(caminho_csv)
                    df['id_experimento'] = nome_pasta
                    dados_totais.append(df)
                except Exception as e:
                    print(f"Erro ao ler o arquivo {caminho_csv}: {e}")
            else:
                pass 

    if not dados_totais:
        print("Nenhum dado de experimento ('resultados.csv') foi encontrado nas subpastas.")
        return None

    df_final = pd.concat(dados_totais, ignore_index=True)
    
    # Cria o nome do arquivo de saída usando o nome do ambiente
    match = re.search(r'groups/([^/]+)', caminho_base)
    ambiente_nome = match.group(1) if match else 'unknown_environment'
    
    caminho_saida = os.path.join(caminho_base, 'plots', f"compilado_{ambiente_nome}.csv")
    
    os.makedirs(os.path.dirname(caminho_saida), exist_ok=True)
    df_final.to_csv(path_or_buf=caminho_saida, mode='w', index=False)
    return df_final, caminho_saida

def load_and_group_data(df):
    """
    Recebe o DataFrame compilado, calcula o 'timestep' por experimento, 
    e então calcula Média e Desvio Padrão para TODAS as métricas.
    """
    if df is None or df.empty:
        return None

    if 'id_experimento' not in df.columns:
        print(f"Erro: Coluna 'id_experimento' não encontrada no DataFrame compilado.")
        return None

    # Garante que 'id_experimento' não seja agrupada
    group_by_cols = [col for col in df.columns if col != 'id_experimento']
    
    # Cria a coluna 'timestep' (época/iteração) para cada experimento
    df['timestep'] = df.groupby('id_experimento').cumcount()
    
    # Agrupa por 'timestep' e calcula a Média e o STD de todas as métricas
    grouped_data = df.groupby('timestep')[group_by_cols].agg(['mean', 'std'])
    
    # Renomeia as colunas para o formato 'métrica_agregação'
    grouped_data.columns = [f'{col[0]}_{col[1]}' for col in grouped_data.columns]
    
    return grouped_data.reset_index()

def load_rewards_from_folders(root_directory, folder_list, rewards_filename='rewards.csv'):
    """
    Lê os rewards.csv, transforma (transpõe) e calcula a Média e STD por timestep.
    """
    all_experiments_df = []
    
    for folder_name in folder_list:
        file_path = os.path.join(root_directory, folder_name, rewards_filename)
        if not os.path.exists(file_path):
            continue
        try:
            # Lê, transpõe e garante que os dados são numéricos
            df_reward = pd.read_csv(file_path, header=0).T
            df_reward = df_reward.apply(pd.to_numeric, errors='coerce')
            df_reward['experiment_id'] = folder_name
            all_experiments_df.append(df_reward)
            
        except Exception as e:
            print(f"Erro ao processar {file_path}: {e}")
            
    if not all_experiments_df:
        return None

    combined_df = pd.concat(all_experiments_df, ignore_index=True)
    data_cols = combined_df.drop(columns=['experiment_id']).dropna(axis=1, how='all')
    
    mean_rewards = data_cols.mean(axis=0)
    std_rewards = data_cols.std(axis=0)
    
    grouped_data = pd.DataFrame({
        'timestep': mean_rewards.index, 
        'mean_reward': mean_rewards.values,
        'std_reward': std_rewards.values
    })
    
    grouped_data['timestep'] = pd.to_numeric(grouped_data['timestep'], errors='coerce')

    return grouped_data

def plot_reward_mean_std(grouped_data, directory):
    """Gera o plot de Recompensa (Média +/- std)."""
    if grouped_data is None or grouped_data.empty:
        print("Aviso: Dados de recompensa agrupados estão vazios. Pulando plotagem de Recompensa.")
        return
        
    plt.figure()
    mean = grouped_data['mean_reward']
    std = grouped_data['std_reward']
    timestep = grouped_data['timestep']
    
    plt.plot(timestep, mean, label='Média de Recompensa', color='blue')
    
    plt.fill_between(
        timestep,
        mean - std, 
        mean + std,   
        alpha=0.2,                     
        color='blue',
    )

    fechar_plot(directory, 'reward_mean_std', 'Iteração/Época', 'Recompensa Média')

def plot_rl_metrics(data_grouped, directory):
    """
    Gera plots de Loss (com componentes), Gradiente e Pesos (média +/- std).
    """
    if data_grouped is None or data_grouped.empty:
        return
        
    metric_map = OrderedDict([
        ('loss', {'title': 'Componentes da Loss', 'ylabel': 'Valor da Loss Média', 'components': ['entropy_loss', 'policy_gradient_loss', 'value_loss']}),
        ('actor_weight', {'title': 'Pesos do Ator', 'ylabel': 'Valor do Peso', 'components': None}),
        ('actor_grad', {'title': 'Gradiente do Ator', 'ylabel': 'Valor do Gradiente', 'components': None}),
        ('critic_weight', {'title': 'Pesos do Crítico', 'ylabel': 'Valor do Peso', 'components': None}),
        ('critic_grad', {'title': 'Gradiente do Crítico', 'ylabel': 'Valor do Gradiente', 'components': None}),
    ])

    for metric_prefix, info in metric_map.items():
        plt.figure()
        
        if info['components']:
            plot_list = info['components']
        else:
            # Encontra todas as colunas que começam com o prefixo e terminam com _mean
            all_means = [col.replace('_mean', '') for col in data_grouped.columns if col.startswith(metric_prefix) and col.endswith('_mean')]
            if not all_means:
                continue
            plot_list = all_means

        for prefix in plot_list:
            mean_col = f'{prefix}_mean'
            std_col = f'{prefix}_std'
            if mean_col not in data_grouped.columns:
                continue

            legend_name = prefix.replace('_', ' ').title()
            plt.plot(data_grouped['timestep'], data_grouped[mean_col], label=legend_name)
            
            if std_col in data_grouped.columns:
                plt.fill_between(
                    data_grouped['timestep'],
                    data_grouped[mean_col] - data_grouped[std_col],
                    data_grouped[mean_col] + data_grouped[std_col],
                    alpha=0.2,
                )

        fechar_plot(directory, metric_prefix, 'Época', info['ylabel'])

def plot_im_subplots(data, name, output_dir):
    """
    Plots todas as combinações de Informação Mútua (IM) em subplots de dispersão.
    """
    if data is None or data.empty: 
        print("Aviso: Dados agrupados para IM estão vazios. Pulando plotagem de IM.")
        return
    
    im_combinations = [
        ('I(X,h_1)', 'I(h_1,h_2)', 'Relação $I(X,h_1) \\times I(h_1,h_2)$'),
        ('I(X,h_1)', 'I(h_1,hat Y)', 'Relação $I(X,h_1) \\times I(h_1,\\hat Y)$'),
        ('I(X,h_2)', 'I(h_2,hat Y)', 'Relação $I(X,h_2) \\times I(h_2,\\hat Y)$'),
        ('I(h_1,h_2)', 'I(h_2,hat Y)', 'Relação $I(h_1,h_h2) \\times I(h_2,\\hat Y)$')
    ]

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 12))
    axes = axes.flatten() 
    lista_letra = ['a)', 'b)', 'c)', 'd)']

    for i, (col_x, col_y, title) in enumerate(im_combinations):
        ax = axes[i] 
        
        col_x_mean = f'{col_x}_mean'
        col_y_mean = f'{col_y}_mean'

        if col_x_mean not in data.columns or col_y_mean not in data.columns:
            ax.set_visible(False) 
            continue
        
        val_x_mean = data[col_x_mean]
        val_y_mean = data[col_y_mean]
        
        scatter = ax.scatter(
            val_x_mean, 
            val_y_mean,
            c=data['timestep'],
            cmap='magma',
        )
        
        # Formatando labels com LaTeX
        x_label = col_x.replace('hat', '\\hat')
        y_label = col_y.replace('hat', '\\hat')

        ax.set_title(title)
        ax.set_xlabel(f'${x_label}$')
        ax.set_ylabel(f'${y_label}$')
        ax.text(0.02,0.98,lista_letra[i], transform=ax.transAxes, va='top')
        
    # Adicionando a Colorbar fora da área principal
    cbar_ax = fig.add_axes([0.95, 0.1, 0.025, 0.8]) # Posição ajustada
    fig.colorbar(scatter, cax=cbar_ax, label= 'Épocas')
    fig.tight_layout(rect=[0, 0, 0.9, 1]) # Ajusta o layout para dar espaço à colorbar

    plots_dir = os.path.join(output_dir, 'plots')

    match = re.search(r'compilado_(.+)\.csv', name)
    if match:
        enviroment = match.group(1)
        plt.savefig(os.path.join(plots_dir, f'Mean_IM_{enviroment}.pdf'))
    else:
        plt.savefig(os.path.join(plots_dir, f'Mean_IM_unknown.pdf'))
        
    plt.close()

def main_pipeline(enviroment_name, root_dir_prefix='../data/groups/'):
    """
    Executa o pipeline completo: coleta, agrupa e plota.
    """
    root_directory = os.path.join(root_dir_prefix, enviroment_name)

    try:
        df_completo, compiled_data_path = coletar_dados_experimentos(root_directory)
    except TypeError:
         # Se a função retornar None, o df_completo será None, e o pipeline para aqui.
         print("Erro na coleta de dados. Terminando o pipeline.")
         return
    
    if df_completo is None:
        return
        
    output_dir = root_directory
    grouped_data_compiled = load_and_group_data(df_completo)
    
    if grouped_data_compiled is not None:
        
        # Plotagem de Loss, Gradiente e Pesos
        try:
            plot_rl_metrics(grouped_data_compiled, output_dir)
        except Exception as e:
            print(f"Ocorreu um erro ao gerar plots de métricas RL: {e}")
            
        # Plotagem de Informação Mútua (IM)
        try:
            plot_im_subplots(grouped_data_compiled, compiled_data_path, output_dir)
        except Exception as e:
            print(f"Ocorreu um erro ao gerar plots de IM: {e}")
            
    try:
        folder_names = [d for d in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, d))]
        if folder_names:
            grouped_rewards = load_rewards_from_folders(root_directory, folder_names)
            if grouped_rewards is not None:
                plot_reward_mean_std(grouped_rewards, output_dir)
        else:
             print(f"\nAviso: Nenhuma subpasta de experimento encontrada em {root_directory}. Pulando plotagem de Recompensa.")
    except FileNotFoundError:
        print(f"\nAviso: Diretório de recompensas {root_directory} não encontrado. Pulando plotagem de Recompensa.")
    except Exception as e:
        print(f"\nOcorreu um erro ao processar plots de Recompensa: {e}")
        


if __name__ == '__main__':
    environment_to_run = 'Acrobot_2x32' 
    main_pipeline(environment_to_run)

/tmp/ipykernel_59224/3818098783.py:260: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0, 0.9, 1]) # Ajusta o layout para dar espaço à colorbar


Plot de Recompensa concluído.
